Vorbereitung der Umgebung Benötigte Packete -nympy -matplotlib -ucimlrepo -scikit-learn

-jupyter-notebook Umgebung (z.B. jupyterlab)

jupyterlab mit "jupyter-lab" in Konsole ausführen

# ###########################################################################################
# HANDS-ON: Bestimme die schnellsten und genauesten ML-Classifier ###########################
# Optimierungsziel: Minimiere Trainingszeit, Maximiere Genaugkeit (accuracy) ################
# ###########################################################################################

In [ ]:
# lade bib für die Beispieldaten
from ucimlrepo import fetch_ucirepo

# lade standard-bibliotheken
import time
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# lade scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# falls ihr zuviele Meldungen beim ausführen bekommt, könnt ihr die Warnings ausschalten
#import warnings
#warnings.filterwarnings("ignore")

In [ ]:
# definiere Funktion zum Auswerten (gerne verbessern)
def calculateAccuracyVsTime(classifiers, n_repetitions, test_size=0.2):
    # n_repetitions -> wir berechnen Mittelwert aus dieser Zahl der Wiederholungen. Falls ihr später einen langsamen classifier erwischt, hier die Anzahl entsprechend runterdrehen
    calculated_values = []

    for classifier_idx, (name, classifier) in enumerate(classifiers.items()):
        start_time = time.time()
        accuracies = []
        for iRep in range(n_repetitions):
            # wähle zufälligen train-test split in jeder Iteration
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
            # trainiere das Modell und berechne predictions (ich musste values.ravel() aufgrund des falschen Shapes einbauen, vllt geht es schöner)
            curr_accuracy = accuracy_score(y_test.values.ravel(), classifier.fit(X_train, y_train.values.ravel()).predict(X_test)) 
            accuracies.append(curr_accuracy * 100.)
        end_time = time.time()
        accuracy = np.mean(accuracies)
        elapsed_time = 1000*(end_time - start_time) / n_repetitions # wir vermischen absichtlich train- und predict-Zeit
        print("Idx: %d, Accuracy (test) for %s: %.1f%%, duration: %.2fms" % (classifier_idx, name, accuracy, elapsed_time ))
        calculated_values.append((name, accuracy, elapsed_time, classifier_idx))


    column_names = ['Model', 'Accuracy', 'Time', 'Idx']
    df = pd.DataFrame(calculated_values, columns=column_names)
    fig, (ax1) = plt.subplots(1,1)

    # just plotting, passt es an, wenn ihr das mit den Farben schneller hinbekommt ;)
    plt.scatter(df['Time'], df['Accuracy'], c = df['Idx'], s=200, cmap='viridis')
    plt.xlabel('Time [ms]')
    plt.ylabel('Accuracy [%]')
    plt.title('Accuracy vs Time')
    plt.colorbar(label='Index')
    fig.set_size_inches(15,10)

    # Falls ihr die Werte in Excel anschauen wollt, hier auskommentieren und output in Excel copy-pasten
    #for sublist in calculated_values:
    #    print(";".join(map(str, sublist)))
    
    return calculated_values

In [ ]:
# fetch dataset
# Schaut auf https://archive.ics.uci.edu/dataset/186/wine+quality die Infos zum Datensatz nach, falls ihr wissen wollt welche Parameter da vorhanden sind

wine_quality = fetch_ucirepo(id=186) #mMn sind hier Rot- und Weißweindaten vermischt, was u.U. die Datenbasis verzerrt, ist aber für unsere Fragestellung nicht so entscheidend  ;)

# data (as pandas dataframes)
X = wine_quality.data.features
y = wine_quality.data.targets

# zur Info die Werteverteilung
print(X.describe())
print(y.describe())

# Aufgabe zur Pareto-Front: Genauigkeit vs. Trainingszeit

In [ ]:
# führt zuerst das Skript aus und schaut das Ergebnis an, Anpassungen und Aufgaben kommen im Anschluss, ihr könnt dann zurück-kehren


# wir definieren hier die classifier, die wir untersuchen möchten. Das Ziel ist es die schnellsten (Trainingszeit) und die genauigsten (accuracy) zu finden.
# Also das Optimierungsziel lautet maximiere Genaugkeit, minimiere Traingszeit

# hier, die ich schon ausprobiert waren, die aber eher schlecht performen (und den Plot unübersichtlich machen), könnt aber gerne sich selbst überzeugen
#    "SVC (ovo)": svm.SVC(decision_function_shape='ovo'),
#    "MLP Classifier 15": MLPClassifier(max_iter=1000, hidden_layer_sizes=(15,)),

classifiers_init = {
 "RFC (10)":  RandomForestClassifier(n_estimators=10),
 "RFC (20)":  RandomForestClassifier(n_estimators=20),
 "DTC (4)":   DecisionTreeClassifier(max_depth=4),
 "DTC (None)":DecisionTreeClassifier(max_depth=None),
 "KNC (10)":  KNeighborsClassifier(n_neighbors=10),
 "KNC (7)":   KNeighborsClassifier(n_neighbors=7),
 "KNC (5)":   KNeighborsClassifier(n_neighbors=5),
 "GNB":       GaussianNB(),
 "NearestCentroid (21)": NearestCentroid(shrink_threshold=21),
}

n_repetitions = 100 # wir berechnen Mittelwert aus dieser Zahl der Wiederholungen. Falls ihr später einen langsamen classifier erwischt, hier die Anzahl entsprechend runterdrehen

calculated_values = calculateAccuracyVsTime(classifiers_init, n_repetitions)


# Aufgaben:
- Welche der dargestellten Punkte (Classifier) bilden die Pareto-Front ?
- Welche der dargestellten Punkte (Classifier) sind nicht optimal im Bezug auf Genaugkeit / Trainingszeit
- Könnt ihr ableiten, mit welchen Parametern (ML-Modellen) man die beste Genauigkeit (je nach Trainingszeit) erzielt ?
- Könnt ihr weitere pareto-optimale classifier finden (Übersichtlichkeits- und vergleichbarkeitshalber den Code copy-pasten und anpassen, um vorherige Plots behalten zu können)
    - Parameteranpassung der bestehenden (Parameter in classifiers anpassen)
    - Andere Classifier: siehe scikit-learn Doku (z.b. https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py oder https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)
        - Erweiterung der classifier-variable "classifiers = {...}" um neue Callsifier. Vergebt entsprechenden Namen um in der Textausgabe besser die Genaugikeit zuordnen zu können
        - Denkt dran die entsprechenden Bibliotheken für den neuen Classifier einzubinden
- (Optional) Was passiert, wenn man test_size=0.2 ändert ? 
- (Optional) Hier wurden Classifier verwendet (Bewertungen: 3,4,5,6,7,8,9). Alternativ könnte man ebenso hier Regressoren (langsamer) verwenden und "Zwischenbewertungen" bestimmen.
- (Optional) Man kann den Test mit einem anderen Datensatz ausprobieren (id ändern fetch_ucirepo(id=186)), schaut vorher in Doku nach ob der Datensatz in Frage käme

In [ ]:
# hier eure angepasste classifier

# from sklearn.xxxxxx import xxxxxx

classifiers_yours = {
 "RFC (10)":  RandomForestClassifier(n_estimators=10),
 "RFC (20)":  RandomForestClassifier(n_estimators=20),
 "DTC (4)":   DecisionTreeClassifier(max_depth=4),
 "DTC (None)":DecisionTreeClassifier(max_depth=None),
 "KNC (10)":  KNeighborsClassifier(n_neighbors=10),
 "KNC (7)":   KNeighborsClassifier(n_neighbors=7),
 "KNC (5)":   KNeighborsClassifier(n_neighbors=5),
 "GNB":       GaussianNB(),
 "NearestCentroid (21)": NearestCentroid(shrink_threshold=21),
 # .... the stage is yours
}
n_repetitions = 100
calculated_values = calculateAccuracyVsTime(classifiers_yours, n_repetitions)